### SegNet

We implement the SegNet architecture ([Badrinarayanan et al. 2015](https://arxiv.org/abs/1511.00561)) using the Keras framework.

![](./img/SegNet_architecture.png)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py

**Load data**

In [ ]:
from load_batches import load_png_3bands, load_png_1band, load_tif_3bands

In [ ]:
# Validation data
x_validation = load_png_3bands("/Users/ikersanchez/Vizzuality/PROIEKTUAK/Skydipper/Data/dataset1/images_prepped_test/*.png")
y_validation = load_png_1band("/Users/ikersanchez/Vizzuality/PROIEKTUAK/Skydipper/Data/dataset1/annotations_prepped_test/*.png")

# Train data
x_train = load_png_3bands("/Users/ikersanchez/Vizzuality/PROIEKTUAK/Skydipper/Data/dataset1/images_prepped_train/*.png")
y_train = load_png_1band("/Users/ikersanchez/Vizzuality/PROIEKTUAK/Skydipper/Data/dataset1/annotations_prepped_train/*.png")

In [ ]:
# Train data
x_train_tif = load_tif_3bands("/Users/ikersanchez/Vizzuality/PROIEKTUAK/Skydipper/Data/Potsdam/2_Ortho_RGB/*_2_10_RGB.tif")
y_train_tif = load_tif_3bands("/Users/ikersanchez/Vizzuality/PROIEKTUAK/Skydipper/Data/Potsdam/5_Labels_all/*_2_10_label.tif")

**Visualize data**

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10,5))
ax = axs[0]
ax.imshow(x_train[0,:,:,:]);

ax = axs[1]
ax.imshow(y_train[0,:,:]);

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10,5))
ax = axs[0]
ax.imshow(x_train_tif[0,:,:,:]);

ax = axs[1]
ax.imshow(y_train_tif[0,:,:,:]);

**Preprocess class labels for Keras**

Let's take a look at the shape of our class label data:

In [ ]:
y_train.shape

The y_train and y_test data are not split into 10 distinct class labels, but rather are represented as a single array with the class values. We can fix this easily:

In [ ]:
from keras.utils import np_utils
# Convert 1-dimensional class arrays to 12-dimensional class matrices
y_train = np_utils.to_categorical(y_train, num_classes=12)
y_validation = np_utils.to_categorical(y_validation, num_classes=12)
y_train.shape

In the y_train_tif data the six categories/classes have been defined as:
    1. Impervious surfaces (RGB: 255, 255, 255)
    2. Building (RGB: 0, 0, 255)
    3. Low vegetation (RGB: 0, 255, 255)
    4. Tree (RGB: 0, 255, 0)
    5. Car (RGB: 255, 255, 0)
    6. Clutter/background (RGB: 255, 0, 0)

In [ ]:
y_train_tif.shape

In [ ]:
def from_rgb_to_categorical(array):
    t = array.shape[0]
    y = array.shape[1]
    x = array.shape[2]
    c = array.shape[3]
    
    image = np.zeros((t,y,x))
    
    # 0. Impervious surfaces (RGB: 255, 255, 255)
    image[np.where((y_train_tif[:,:,:,0] == 1.) & (y_train_tif[:,:,:,1] == 1.) & (y_train_tif[:,:,:,2] == 1.))] = 0.
    # 1. Building (RGB: 0, 0, 255)
    image[np.where((y_train_tif[:,:,:,0] == 0.) & (y_train_tif[:,:,:,1] == 0.) & (y_train_tif[:,:,:,2] == 1.))] = 1.
    # 2. Low vegetation (RGB: 0, 255, 255)
    image[np.where((y_train_tif[:,:,:,0] == 0.) & (y_train_tif[:,:,:,1] == 1.) & (y_train_tif[:,:,:,2] == 1.))] = 2.
    # 3. Tree (RGB: 0, 255, 0)
    image[np.where((y_train_tif[:,:,:,0] == 0.) & (y_train_tif[:,:,:,1] == 1.) & (y_train_tif[:,:,:,2] == 0.))] = 3.
    # 4. Car (RGB: 255, 255, 0)
    image[np.where((y_train_tif[:,:,:,0] == 1.) & (y_train_tif[:,:,:,1] == 1.) & (y_train_tif[:,:,:,2] == 0.))] = 4.
    # 5. Clutter/background (RGB: 255, 0, 0)
    image[np.where((y_train_tif[:,:,:,0] == 1.) & (y_train_tif[:,:,:,1] == 0.) & (y_train_tif[:,:,:,2] == 0.))] = 5.
    
    # Convert 1-dimensional class arrays to 6-dimensional class matrices
    image = np_utils.to_categorical(image, num_classes=6)
    return image

In [ ]:
y_train_tif = from_rgb_to_categorical(y_train_tif)

In [ ]:
y_train_tif.shape

**Save samples**

In [ ]:
f = h5py.File("./Samples/data1/x_train.h5", 'w')
f.create_dataset('x_train', data=x_train)     
f.close()

f = h5py.File("./Samples/data1/y_train.h5", 'w')
f.create_dataset('y_train', data=y_train)     
f.close()

f = h5py.File("./Samples/data1/x_validation.h5", 'w')
f.create_dataset('x_validation', data=x_validation)     
f.close()

f = h5py.File("./Samples/data1/y_validation.h5", 'w')
f.create_dataset('y_validation', data=y_validation)     
f.close()              

## Train de Network 

In [ ]:
from train_SegNet import LossHistory, train_segnet

In [ ]:
root_in = "Samples/data1/"
root_out = "Network/SegNet"
nEpochs = 2
option = 'start'

out = train_segnet(root_in, root_out, option)
    
out.read_data()

if (option == 'start'):           
    out.define_network()        
        
if (option == 'continue'):
    out.read_network()

if (option == 'start' or option == 'continue'):
    out.compile_network()
    out.train_network(nEpochs)

## Predict

In [ ]:
from SegNet import segnet

In [ ]:
input_path = 'Samples/x_validation.h5'
output_path = 'Predictions/prediction.h5'

# Open file with observations and read them. We use h5 in our case
f = h5py.File(input_path, 'r')
imgs = f.get("x_validation")
f.close()  
    
prediction = segnet(imgs, output_path)
prediction.define_network()
out = prediction.predict()